In [4]:
import requests
from bs4 import BeautifulSoup
import time, random
import re
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_0) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
}
list_href_player=[]
year_list=[2025,2024,2023,2022,2021,2020]
i=0
url_champion=["https://www.basketball-reference.com/teams/OKC/2025.html","https://www.basketball-reference.com/teams/BOS/2024.html","https://www.basketball-reference.com/teams/DEN/2023.html","https://www.basketball-reference.com/teams/GSW/2022.html","https://www.basketball-reference.com/teams/MIL/2021.html","https://www.basketball-reference.com/teams/LAL/2020.html"]
for url in url_champion:

    response = requests.get(url, headers=headers)
    response.encoding = "utf-8"
    print(response.status_code)
    time.sleep(random.uniform(3,5))
    soup = BeautifulSoup(response.text, "html.parser")
    rows = soup.select("table#per_game_stats tbody tr")

    for row in rows:

        player_cell = row.select("td")
        link_player = player_cell[0].select_one("a")
        age=player_cell[1].text
        age_int=int(age)

        print(link_player["href"])

        list_href_player.append({"year":year_list[i], "url_player": link_player["href"], "age": age_int, "player_rank":None,"total_point":None})

    i=i+1

200
/players/g/gilgesh01.html
/players/w/willija06.html
/players/d/dortlu01.html
/players/h/harteis01.html
/players/w/wallaca01.html
/players/h/holmgch01.html
/players/w/wiggiaa01.html
/players/j/joeis01.html
/players/c/carusal01.html
/players/w/willija07.html
/players/m/mitchaj01.html
/players/w/willike04.html
/players/d/diengou01.html
/players/j/jonesdi01.html
/players/c/carlsbr01.html
/players/d/ducasal01.html
/players/f/flaglad01.html
/players/l/leonsma01.html
/players/r/reeseal01.html
200
/players/t/tatumja01.html
/players/b/brownja02.html
/players/h/holidjr01.html
/players/w/whitede01.html
/players/p/porzikr01.html
/players/h/horfoal01.html
/players/p/pritcpa01.html
/players/h/hausesa01.html
/players/k/kornelu01.html
/players/t/tillmxa01.html
/players/q/quetane01.html
/players/b/brissos01.html
/players/m/mykhasv01.html
/players/w/walshjo01.html
/players/p/peterdr01.html
/players/s/sprinja01.html
/players/b/bantoda01.html
/players/s/stevela01.html
/players/d/davisjd01.html
200
/pl

In [18]:
import pandas as pd
positions = []
records=[]
height_cm=0
weight_kg=0
experience_years=0
cleaned=None
player_name=None
team_name=None
base_url = "https://www.basketball-reference.com"
for player in list_href_player:
    response = requests.get(base_url+player["url_player"],headers=headers)
    response.encoding = "utf-8"
    info_player_soup = BeautifulSoup(response.text, "html.parser")

    url_player = player["url_player"]  # just a string
    player_id = re.search(r'([^/]+)\.html$', url_player).group(1)

    player_name_elem = info_player_soup.select_one("#meta h1")
    player_name = player_name_elem.get_text(strip=True) if player_name_elem else None
    print(player_name)

    final_clean_positions = []
    position_elem = info_player_soup.select_one("#meta p:has(strong:-soup-contains('Position'))")

    text = position_elem.get_text(" ", strip=True)
    position = text.split("Shoots:")[0].replace("Position:", "").strip()
    clean_position = position.replace(", and", ",")
    clean2_position = clean_position.replace(" and ", ",")
    positions = [p.strip() for p in clean2_position.split(",") if p.strip()]
    for p in positions:
        cleaned = p.strip()
        cleaned = cleaned.replace("▪", "")
        cleaned = cleaned.strip()
        final_clean_positions.append(cleaned)

    team_strong = info_player_soup.select_one("#meta strong:-soup-contains('Team')")
    if team_strong:
        team_link_tag = team_strong.find_next("a")
        if team_link_tag:
            team_name = team_link_tag.get_text(strip=True)
    else:
        team_name = None



    exp_strong = info_player_soup.select_one("#meta strong:-soup-contains('Experience')")
    if exp_strong:
        experience_text = exp_strong.next_sibling.strip()
        match = re.search(r"\d+", experience_text)
        experience_years = int(match.group(0))

    else:
        exp_strong = info_player_soup.select_one("#meta strong:-soup-contains('Career Length')")
        experience_text = exp_strong.next_sibling.strip()
        match = re.search(r"\d+", experience_text)
        experience_years = int(match.group(0))




    height_weight_elems = info_player_soup.select_one("#meta p:has(span:-soup-contains('lb'))")
    text = height_weight_elems.get_text(" ", strip=True)
    match = re.search(r"\((\d+)cm,\s*(\d+)kg\)", text)
    height_cm = int(match.group(1))
    weight_kg = int(match.group(2))

    records.append({
        "player_id": player_id,
        "player_name": player_name,
        "age": player["age"],
        "rank": player["player_rank"],
        "season": player["year"],
        "height_cm": height_cm,
        "weight_kg": weight_kg,
        "experience_years": experience_years,
        "positions": final_clean_positions,
        "team": team_name,
        "total_point": player["total_point"],
        "groupName":"LC"
    })

    time.sleep(random.uniform(3,5))
df3 = pd.DataFrame(records)

Shai Gilgeous-Alexander
Jalen Williams
Luguentz Dort
Isaiah Hartenstein
Cason Wallace
Chet Holmgren
Aaron Wiggins
Isaiah Joe
Alex Caruso
Jaylin Williams
Ajay Mitchell
Kenrich Williams
Ousmane Dieng
Dillon Jones
Branden Carlson
Alex Ducas
Adam Flagler
Malevy Leons
Alex Reese
Jayson Tatum
Jaylen Brown
Jrue Holiday
Derrick White
Kristaps Porziņģis
Al Horford
Payton Pritchard
Sam Hauser
Luke Kornet
Xavier Tillman Sr.
Neemias Queta
Oshae Brissett
Svi Mykhailiuk
Jordan Walsh
Drew Peterson
Jaden Springer
Dalano Banton
Lamar Stevens
JD Davison
Nikola Jokić
Jamal Murray
Kentavious Caldwell-Pope
Aaron Gordon
Michael Porter Jr.
Bruce Brown
Reggie Jackson
Jeff Green
Bones Hyland
Christian Braun
DeAndre Jordan
Vlatko Čančar
Zeke Nnaji
Thomas Bryant
Ish Smith
Davon Reed
Peyton Watson
Jack White
Stephen Curry
Andrew Wiggins
Jordan Poole
Klay Thompson
Draymond Green
Otto Porter Jr.
Kevon Looney
Damion Lee
Andre Iguodala
Gary Payton II
Jonathan Kuminga
Nemanja Bjelica
Juan Toscano-Anderson
Moses Moody


In [19]:
df3

,player_id,player_name,age,rank,season,height_cm,weight_kg,experience_years,positions,team,total_point,groupName
0,gilgesh01,Shai Gilgeous-Alexander,26,None,2025,198,90,7,"[Point Guard, Shooting Guard]",Oklahoma City Thunder,None,LC
1,willija06,Jalen Williams,23,None,2025,198,99,3,"[Shooting Guard, Power Forward]",Oklahoma City Thunder,None,LC
2,dortlu01,Luguentz Dort,25,None,2025,193,99,6,"[Small Forward, Shooting Guard]",Oklahoma City Thunder,None,LC
3,harteis01,Isaiah Hartenstein,26,None,2025,213,115,7,"[Center, Power Forward]",Oklahoma City Thunder,None,LC
4,wallaca01,Cason Wallace,21,None,2025,193,92,2,[Shooting Guard],Oklahoma City Thunder,None,LC
...,...,...,...,...,...,...,...,...,...,...,...,...
110,danietr01,Troy Daniels,28,None,2020,193,90,7,[Shooting Guard],None,None,LC
111,cacokde01,Devontae Cacok,23,None,2020,201,108,3,"[Power Forward, Center]",None,None,LC
112,dudleja01,Jared Dudley,34,None,2020,198,107,14,"[Power Forward, Small Forward, Shooting Guard]",None,None,LC
113,antetko01,Kostas Antetokounmpo,22,None,2020,208,90,3,"[Power Forward, Small Forward]",None,None,LC


In [20]:
df3.to_csv("championPlayer.csv", index=False)